In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools
import json
from pathlib import Path
from datetime import timedelta, datetime
from scipy.sparse import csr_matrix, vstack


import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [12]:
# Path("/media/hpi_share/")#Path("//FS23/projekte$/MP2021/MPWS2021/MPWS2021FN1")
mp_drive_dir = Path("../../../")
mp_plot_dir = mp_drive_dir / "plots"
input_path = Path("../../matched-infoboxes-extracted/")
input_data = list(input_path.rglob("*.json"))
files = [x for x in input_data if x.is_file()]
len(files)  # total 580


70

In [4]:
# num_edits = 0
# change_tuples = []
# #np.random.seed(42)
# for file in tqdm(np.random.choice(files, 70, replace=False)):
#     with open(file, 'r', encoding='utf-8') as f:
#         for jsonObj in f:
#             single_edit = json.loads(jsonObj)
#             #entries.append(single_edit)
#             key = single_edit['key']
#             pid = single_edit['pageID']
#             user = single_edit['username'] if 'username' in single_edit.keys() else None
#             title = single_edit['pageTitle']#['key']
#             changes = single_edit['changes']
#             timestamp = single_edit['validFrom']
#             edit_type = single_edit['type']
#             for change in changes:
#                 name = change['property']['name']
#                 if 'currentValue' in change.keys() and 'previousValue' in change.keys():
#                     current_value = change['currentValue'] if 'currentValue' in change.keys() else None
#                     previous_value = change['previousValue'] if 'previousValue' in change.keys() else None
#                     if current_value != '' and previous_value != '':
#                         change_tuples.append((key, pid, title, user, name, previous_value, current_value, timestamp, edit_type))

In [13]:
num_iteratons = 1
number_of_files =5
number_of_files_start = 5
num_edits = 0
num_change_tuples = 0
typo_lst = []
for _ in range(num_iteratons):
    change_tuples = []
    for file in tqdm(files[number_of_files_start:number_of_files_start+number_of_files]):
        with open(file, 'r', encoding='utf-8') as f:
            for jsonObj in f:
                single_edit = json.loads(jsonObj)
                num_edits += 1
                title = single_edit['pageTitle']
                pageID = single_edit['pageID']
                key = single_edit['key']
                template = single_edit['template'] if 'template' in single_edit.keys(
                ) else None
                changes = single_edit['changes']
                validFrom = single_edit['validFrom']
                revisionId = single_edit['revisionId']
                attributes = single_edit['attributes'] if 'attributes' in single_edit.keys(
                ) else None
                user_name = single_edit['user']['username'] if 'username' in single_edit['user'].keys(
                ) else None
                user_id = single_edit['user']['id'] if 'id' in single_edit['user'].keys(
                ) else None
                user_ip = single_edit['user']['ip'] if 'ip' in single_edit['user'].keys(
                ) else None
                for change in changes:
                    num_change_tuples += 1
                    name = change['property']['name']
                    current_value = change['currentValue'] if 'currentValue' in change.keys(
                    ) else None
                    previous_value = change['previousValue'] if 'previousValue' in change.keys(
                    ) else None
                    validTo = change['valueValidTo'] if 'valueValidTo' in change.keys(
                    ) else None
                    change_tuples.append((title, pageID, key, template, name, previous_value,
                                          current_value, validFrom, validTo, revisionId, user_name, user_id, user_ip, attributes))
    number_of_files_start += number_of_files

  0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
# data = pd.DataFrame(change_tuples, columns=['key', 'pageID', 'title', 'user','name', 'previous_value', 'current_value', 'timestamp', 'edit_type'])
# data['timestamp'] = pd.to_datetime(data['timestamp']).dt.tz_localize(None)
data = pd.DataFrame(change_tuples, columns=['pageTitle', 'pageID', 'key', 'template', 'name', 'previous_value',
                    'current_value', 'validFrom', 'validTo', 'revisionId', 'user_name', 'user_id', 'user_ip', 'attributes'])
data['validFrom'] = pd.to_datetime(data['validFrom'])
data['validTo'] = pd.to_datetime(data['validTo'])

In [17]:
data.head(10)

,pageTitle,pageID,key,template,name,previous_value,current_value,validFrom,validTo,revisionId,user_name,user_id,user_ip,attributes
0,Indonesian National Armed Forces,14650,119306865-0,infobox national military,reaching,None,"2,201,047",2007-03-31 18:02:53+00:00,2008-07-25 09:13:54+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
1,Indonesian National Armed Forces,14650,119306865-0,infobox national military,image,None,{{border|[[Image:Flag of Indonesia.svg|110px]]}},2007-03-31 18:02:53+00:00,2007-06-21 05:23:24+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
2,Indonesian National Armed Forces,14650,119306865-0,infobox national military,available,None,"60,543,028",2007-03-31 18:02:53+00:00,2010-02-25 13:42:35+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
3,Indonesian National Armed Forces,14650,119306865-0,infobox national military,percent_gdp,None,3%,2007-03-31 18:02:53+00:00,2007-08-30 21:00:35+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
4,Indonesian National Armed Forces,14650,119306865-0,infobox national military,exports,None,,2007-03-31 18:02:53+00:00,2010-02-25 13:42:35+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
5,Indonesian National Armed Forces,14650,119306865-0,infobox national military,reserve,None,,2007-03-31 18:02:53+00:00,2009-08-30 12:47:38+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
6,Indonesian National Armed Forces,14650,119306865-0,infobox national military,age,None,,2007-03-31 18:02:53+00:00,2010-02-25 13:42:35+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
7,Indonesian National Armed Forces,14650,119306865-0,infobox national military,domestic_suppliers,None,,2007-03-31 18:02:53+00:00,2009-03-28 17:56:19+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
8,Indonesian National Armed Forces,14650,119306865-0,infobox national military,history,None,,2007-03-31 18:02:53+00:00,2010-02-25 13:42:35+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."
9,Indonesian National Armed Forces,14650,119306865-0,infobox national military,manpower_data,None,2005 est.,2007-03-31 18:02:53+00:00,2008-01-26 05:55:31+00:00,119306865,Octane,986956.0,None,"{'country': '[[Indonesia]]', 'domestic_supplie..."


--------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
data[data[['pageTitle', 'name', 'timestamp']].duplicated(keep=False)].sort_values(['pageTitle', 'name', 'timestamp']).head(50)

In [ ]:
bins = pd.date_range(data['timestamp'].min().date(), data['timestamp'].max().date()+timedelta(1))
total_days = len(bins)
bins = pd.cut(data['timestamp'], bins, labels=False)
data['bin_idx'] = bins

In [ ]:
def create_time_series(a, duration):
    series = np.zeros(duration)
    uniques, counts = np.unique(a, return_counts=True)
    series[uniques] = counts
    return csr_matrix(series)

def create_bool_time_series(a, duration):
    series = np.zeros(duration, dtype=bool)
    uniques, counts = np.unique(a, return_counts=True)
    series[uniques] = 1
    return series

In [ ]:
print(f"Num data points: {len(data)}")

In [ ]:
num_required_changes = 10
groups = data.groupby(['key', 'name'])
min_support_groups = data[groups['bin_idx'].transform('count') > num_required_changes].groupby(['key', 'name'])

In [ ]:
%%time
min_support_groups = min_support_groups['bin_idx'].apply(create_time_series, duration=total_days)

In [ ]:
print(f"Number of properties: {len(groups)}")
group_index = min_support_groups.index
#x = groups.loc["100593997-0"].values
x = vstack(min_support_groups.to_numpy())
print(f"Len min support groups : {x.shape[0]}")
#x = csr_matrix(x)

## Nearest Neighbors
Probably more suited than clustering methods

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
max_dist = 2
neigh = NearestNeighbors(radius=max_dist, p=1)
neigh.fit(x)

In [ ]:
%%time
dist, ind = neigh.radius_neighbors()

In [ ]:
a = group_index.values
same_infobox = []
for i, indices in enumerate(ind):
    infobox = group_index[i][0]
    if len(indices) > 0:
        matching_pages = np.array([key[0] for key in group_index[indices]])
        same_infobox.append(infobox == matching_pages)
n_total = len(ind)
n_matched = len(same_infobox)

In [ ]:
print(f"number of pages considered: {data['pageID'].nunique()}")

In [ ]:
num_matches = []
percent_same_matched = []
max_num_matches = 100
for i in range(1, max_num_matches):
    num_matches.append(np.sum(np.fromiter( (len(a) for a in same_infobox), dtype=int) == i))
    percent_same_matched.append(np.mean([np.sum(entry) / i for entry in same_infobox if len(entry) == i]))
percent_same_infobox = np.multiply(num_matches, percent_same_matched)
plt.bar(range(1, max_num_matches), percent_same_infobox, label='Same infobox')
plt.bar(range(1, max_num_matches), num_matches-percent_same_infobox, bottom = percent_same_infobox, label='Different Infobox')
plt.legend(loc='upper right')
plt.title(f"Number of properties with #matches == x (#considered: {len(groups)})")
plt.figtext(0.1, 0, "Color represents the % of matches that are not from the same infobox")

In [ ]:
#0.963084495488105
# 95 93 87 87 95
# 93 91 84 93 87
# 94 91 80 90 88
percent_same_matched

Look at some of the examples
 - Many examples have just creation / deletion
 - When filtering these out, some examples are:
     - batting averages (247568784-0), box2..., Statistics in general
     - Temperature reading for cities (infoboxes that are basically plots) - https://de.wikipedia.org/wiki/Ottawa
     - Uniforms for soccer games and stuff https://en.wikipedia.org/wiki/FC_Bayern_Munich_(women)
     - Perfect synchronous data https://en.wikipedia.org/wiki/Delta_County_Airport
     - Career listings and other list elements - https://en.wikipedia.org/wiki/Akaki_Khubutia
     - Stuff that is updated real time, once (Elections https://en.wikipedia.org/wiki/2019_European_Parliament_election_in_the_United_Kingdom)
     - railway stations , passenger numbers (https://en.wikipedia.org/wiki/Windsor_station_(Vermont))
     - Vandalism

In [ ]:
a = group_index[ind[np.array([len(g) for g in ind]) > 2][70]]
print(a)
a = a[6]
data[(data['name'] == a[1]) & (data['key'] == a[0])]

In [ ]:
data[data['key'] == '131458332-1']

In [ ]:
data[data['key'] == '131458332-2']

In [ ]:
data[(data['name'] == 'ability') & (data['key'] == '131458332-2')]

In [ ]:
ind[np.array([len(g) for g in ind]) > 20]

In [ ]:
group_index[ind[np.array([len(g) for g in ind]) > 20][123]]
data[(data['key'] == '110474295-0') & (data['name'] == 'area_rank')]

## KShape, apparently good fast clustering
Seems to be ill-suited for the job

In [ ]:
plt.figure(figsize=(100,2))
for k, row in groups.loc["100593997-0"].items():
    plt.plot(row)

In [ ]:
from tslearn.clustering import KShape
n_clusters = 4
classifier = KShape(n_clusters)
classifier.fit(x)
predictions = classifier.predict(x)

In [ ]:
for i in range(n_clusters):
    selected_data = x[predictions==i]
    plt.figure(figsize=(100,2))
    for j in range(len(selected_data)):
        plt.plot(selected_data[j])
    plt.show()

## DBScan

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
from dtaidistance import dtw

In [ ]:
print(dtw.distance.__doc__)

In [ ]:
def change_distance(a, b):
    return dtw.distance(a, b, window=14, use_pruning=True, use_c=True)

In [ ]:
x.shape

In [ ]:
%%time
clustering = DBSCAN(eps=1, min_samples=2, n_jobs=6, metric=change_distance).fit(x)

In [ ]:
clusters = np.unique(clustering.labels_)
n_clusters = len(clusters)
if -1 in clusters:
    n_clusters -=1

In [ ]:
for i in range(n_clusters):
    selected_data = x[clustering.labels_==i]
    plt.figure(figsize=(100,2))
    for j in range(len(selected_data)):
        plt.plot(selected_data[j])
    plt.show()